<a href="https://colab.research.google.com/github/arinaaandreeva/Network_analysis/blob/main/Clean_MCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import networkx as nx
import heapq
!pip install pulp -q
import pulp
import pandas as pd
import numpy as np
from collections import defaultdict, deque
import time
from typing import List, Dict


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 68.6 MB/s eta 0:00:00


In [2]:
!unzip /content/anonymised_larger_transfer_limits.zip -d my_data

Archive:  /content/anonymised_larger_transfer_limits.zip
   creating: my_data/anonymised/
  inflating: my_data/anonymised/.DS_Store  
  inflating: my_data/__MACOSX/anonymised/._.DS_Store  
   creating: my_data/anonymised/20_nodes/
   creating: my_data/anonymised/50_nodes/
   creating: my_data/anonymised/10_nodes/
   creating: my_data/anonymised/140_nodes/
  inflating: my_data/anonymised/20_nodes/distance_matrix.csv  
  inflating: my_data/anonymised/20_nodes/reqs.csv  
  inflating: my_data/anonymised/20_nodes/offices.csv  
  inflating: my_data/anonymised/50_nodes/distance_matrix.csv  
  inflating: my_data/anonymised/50_nodes/reqs.csv  
  inflating: my_data/anonymised/50_nodes/offices.csv  
  inflating: my_data/anonymised/10_nodes/distance_matrix.csv  
  inflating: my_data/anonymised/10_nodes/reqs.csv  
  inflating: my_data/anonymised/10_nodes/offices.csv  
  inflating: my_data/anonymised/140_nodes/distance_matrix.csv  
  inflating: my_data/anonymised/140_nodes/reqs.csv  
  inflating: my

In [ ]:
# reqs = pd.read_csv('/content/my_data/anonymised/10_nodes/reqs.csv',
#                     dtype={'src_office_id': 'int64',
#                            'dst_office_id': 'int64',
#                            'volume': 'int64'})

# office = pd.read_csv('/content/my_data/anonymised/10_nodes/offices.csv',
#                       dtype={'office_id': 'int64',
#                              'transfer_price': 'float64'})

# dist = pd.read_csv('/content/my_data/anonymised/10_nodes/distance_matrix.csv',
#                     dtype={'src': 'int64',
#                            'dst': 'int64',
#                            'distance': 'float64',
#                            'price': 'float64'})

Постановка задачи:
Дано:

 * Ориентированный граф $ G = (V, E) $
 * Множество товаров (commodities) $ K $
 * Для каждого товара $ k \in K $: источник $ s_k $, сток $ t_k $, требуемый поток $ d_k $
 * Для каждого ребра $ e \in E $: пропускная способность $ u_e $, стоимость использования одним ТС $ c_e $, вместимость ТС $ C $
 * Для каждого узла $ v \in V $:
  - $ f_v \in \mathbb{R}_+ $ — стоимость перегруза единицы груза в узле $ v $
  - $ U_v^- \in \mathbb{R}_+ $ — максимальная пропускная способность по - входящим потокам
  - $ U_v^+ \in \mathbb{R}_+ $ — максимальная пропускная способность по исходящим потокам

Переменные:
 * $ x^k_e \in \mathbb{R}_+ $ — поток товара $ k $ через ребро $ e \in E $
 * $ y_e \in \mathbb{Z}_+ $ — количество ТС, используемых на ребре $ e \in E $

 Целевая функция:

$$ \min \left( \sum_{e \in E} c_e \cdot y_e + \sum_{k \in K} \sum_{p \in \Pi_k} \sum_{\substack{v \in p \\ v \neq s_k \\ v \neq t_k}} f_v \cdot x_p^k \right)
$$

---

## Ограничения:

1. **Полный поток для каждого товара:**

$$ \sum_{p \in \Pi_k} x_p^k = d_k \quad \forall k \in K
$$


2. **Связь между потоком и количеством ТС:**

$$ \sum_{k \in K} \sum_{p \in \Pi_k: e \in p} x_p^k \leq C \cdot y_e \quad \forall e \in E, \quad y_e \in \mathbb{Z}_+
$$

*Вместимость ТС = С = 90*

3. **Неотрицательность потоков:**

$$ x_p^k \geq 0 \quad \forall k \in K, \, \forall p \in \Pi_k$$

4. **Максимальная перегрузка в узле**

$$\sum_{k \in K} \sum_{p \in \Pi_k: v \in p} \sum_{\substack{v \in p \\ v \neq s_k \\ v \neq t_k}} x_p^k \leq U_v \quad \forall v \in V
$$

*Макс перегруз = max_transfer*

In [ ]:
def solve_transportation_problem(reqs, offices, dist, vehicle_capacity=90, max_iterations=5):
    """
    Column Generation для транспортной задачи
    """
    all_offices = offices['office_id'].tolist()

    # кэши для быстрого доступа
    cost_cache = {}
    transfer_prices = {}
    node_capacities = {}

    for _, row in dist.iterrows():
        cost_cache[(row['src'], row['dst'])] = row['price']

    for _, row in offices.iterrows():
        office_id = row['office_id']
        transfer_prices[office_id] = row['transfer_price']
        node_capacities[office_id] = row['transfer_max']

    for office in all_offices:
        cost_cache[(office, office)] = 0.0

    def get_cost(src, dst):
        return cost_cache.get((src, dst), float('inf'))

    def create_path_column(req_idx, path, volume):
        """Создает описание пути"""
        vehicle_cost = sum(get_cost(path[i], path[i+1]) for i in range(len(path)-1))
        transfer_cost = sum(transfer_prices.get(node, 0) for node in path[1:-1])
        total_cost = (vehicle_cost + transfer_cost) * volume

        return {
            'req_idx': req_idx,
            'path': path,
            'cost': total_cost,
            'volume': volume,
            'arcs_used': [(path[i], path[i+1]) for i in range(len(path)-1)],
            'transit_nodes': path[1:-1],
            'src': path[0],
            'dst': path[-1]
        }

    def get_path_signature(path_dict):
        return f"{path_dict['req_idx']}_{'-'.join(map(str, path_dict['path']))}"

    def generate_initial_columns():
        """Генерируем начальные пути"""
        columns = []
        unique_paths = set()

        for idx, row in reqs.iterrows():
            src, dst, volume = row['src_office_id'], row['dst_office_id'], row['volume']

            if src == dst:
                continue

            # Прямой путь
            if get_cost(src, dst) < float('inf'):
                path = create_path_column(idx, [src, dst], volume)
                path_sig = get_path_signature(path)
                if path_sig not in unique_paths:
                    unique_paths.add(path_sig)
                    columns.append(path)

            # через хабы
            hub_candidates = []
            for hub in all_offices:
                if (hub != src and hub != dst and
                    get_cost(src, hub) < float('inf') and get_cost(hub, dst) < float('inf')):
                    total_cost = get_cost(src, hub) + get_cost(hub, dst) + transfer_prices.get(hub, 0)
                    hub_candidates.append((hub, total_cost))

            hub_candidates.sort(key=lambda x: x[1])
            for hub, _ in hub_candidates[:2]:
                path = create_path_column(idx, [src, hub, dst], volume)
                path_sig = get_path_signature(path)
                if path_sig not in unique_paths:
                    unique_paths.add(path_sig)
                    columns.append(path)

        return columns

    def solve_restricted_master(columns):
        """Решам ограниченную основную задачу"""
        prob = pulp.LpProblem("Master", pulp.LpMinimize)

        # Переменные для путей
        path_vars = {i: pulp.LpVariable(f"path_{i}", 0, 1) for i in range(len(columns))}

        # Переменные для ТС
        used_edges = set()
        for col in columns:
            used_edges.update(col['arcs_used'])
        vehicle_vars = {edge: pulp.LpVariable(f"veh_{edge[0]}_{edge[1]}", 0, None, pulp.LpInteger)
                       for edge in used_edges}

        # Целевая функция
        total_cost = pulp.lpSum(col['cost'] * path_vars[i] for i, col in enumerate(columns))
        total_cost += pulp.lpSum(get_cost(*edge) * vehicle_vars[edge] for edge in used_edges)
        prob += total_cost


        for req_idx in reqs.index:
            src, dst = reqs.loc[req_idx, ['src_office_id', 'dst_office_id']]
            if src == dst:
                continue
            relevant = [i for i, col in enumerate(columns) if col['req_idx'] == req_idx]
            if relevant:
                prob += pulp.lpSum(path_vars[i] for i in relevant) == 1, f"demand_{req_idx}"

        # Ограничения ТС
        for edge in used_edges:
            flow = pulp.lpSum(col['volume'] * path_vars[i]
                            for i, col in enumerate(columns) if edge in col['arcs_used'])
            prob += flow <= vehicle_capacity * vehicle_vars[edge], f"cap_{edge[0]}_{edge[1]}"

        # Ограничения узлов
        used_nodes = set()
        for col in columns:
            used_nodes.update(col['transit_nodes'])
        for node in used_nodes:
            if node in node_capacities:
                throughput = pulp.lpSum(col['volume'] * path_vars[i]
                                      for i, col in enumerate(columns) if node in col['transit_nodes'])
                prob += throughput <= node_capacities[node], f"node_{node}"
        prob.solve(pulp.PULP_CBC_CMD(msg=0))

        if prob.status != pulp.LpStatusOptimal:
            return None, None, None, None

        # Двойственные переменные
        duals = {'demand': {}, 'capacity': {}, 'node_capacity': {}}

        for req_idx in reqs.index:
            constraint = prob.constraints.get(f"demand_{req_idx}")
            if constraint:
                duals['demand'][req_idx] = constraint.pi

        for edge in used_edges:
            constraint = prob.constraints.get(f"cap_{edge[0]}_{edge[1]}")
            if constraint:
                duals['capacity'][edge] = constraint.pi

        for node in used_nodes:
            constraint = prob.constraints.get(f"node_{node}")
            if constraint:
                duals['node_capacity'][node] = constraint.pi

        solution = {i: pulp.value(path_vars[i]) for i in path_vars}
        vehicles = {edge: pulp.value(var) for edge, var in vehicle_vars.items()}

        return pulp.value(prob.objective), solution, duals, vehicles

    def solve_pricing(duals, columns):
        """поиск выгодных путей"""
        if not duals:
            return []

        new_columns = []
        unique_paths = set(get_path_signature(col) for col in columns)

        for req_idx, row in reqs.iterrows():
            src, dst, volume = row['src_office_id'], row['dst_office_id'], row['volume']

            if src == dst:
                continue

            best_cost = 0
            best_path = None
            candidates = [] #варианты путей

            # Прямой путь
            if get_cost(src, dst) < float('inf'):
                candidates.append([src, dst])

            # через хабы
            hub_candidates = []
            for hub in all_offices:
                if (hub != src and hub != dst and
                    get_cost(src, hub) < float('inf') and get_cost(hub, dst) < float('inf')):
                    total_cost = get_cost(src, hub) + get_cost(hub, dst) + transfer_prices.get(hub, 0)
                    hub_candidates.append((hub, total_cost))

            hub_candidates.sort(key=lambda x: x[1])
            for hub, _ in hub_candidates[:5]:
                candidates.append([src, hub, dst])

            for path in candidates:
                col = create_path_column(req_idx, path, volume)

                # Приведенная стоимость
                reduced_cost = col['cost']
                reduced_cost -= duals['demand'].get(req_idx, 0) * volume
                for arc in col['arcs_used']:
                    reduced_cost -= duals['capacity'].get(arc, 0) * volume
                for node in col['transit_nodes']:
                    reduced_cost -= duals['node_capacity'].get(node, 0) * volume

                if reduced_cost < best_cost - 1e-5:
                    best_cost = reduced_cost
                    best_path = col

            if best_path and best_cost < -1e-5:
                path_sig = get_path_signature(best_path)
                if path_sig not in unique_paths:
                    new_columns.append(best_path)
                    unique_paths.add(path_sig)

        return new_columns

    # алгоритм Column Generation
    columns = generate_initial_columns()
    best_solution = None
    best_objective = float('inf')

    for iteration in range(max_iterations):
        result = solve_restricted_master(columns)
        if not result or result[0] is None:
            break

        objective, solution, duals, vehicles = result

        if objective < best_objective:
            best_objective = objective
            best_solution = {
                'objective': objective,
                'solution': solution,
                'columns': columns.copy(),
                'vehicles': vehicles
            }

        new_columns = solve_pricing(duals, columns)
        if not new_columns:
            break

        columns.extend(new_columns)

        if len(columns) > 500: #ограничение на кол-во столбцов для более быстрого решения
            break

    return best_solution

def save_solution(solution, filename):
    results = []
    for i, col in enumerate(solution['columns']):
        usage = solution['solution'].get(i, 0)
        if usage > 1e-6:
            volume = col['volume'] * usage
            path_nodes = [int(node) for node in col['path']]

            results.append({
                'src': int(col['src']),
                'dst': int(col['dst']),
                'volume': float(volume),
                'path_nodes': path_nodes
            })

    pd.DataFrame(results).to_csv(filename, index=False)

def run_for_all_folders():
    """Запуск"""
    base_dir = '/content/my_data/anonymised'
    folders = ['10_nodes', '20_nodes', '50_nodes', '140_nodes']

    for folder in folders:
      reqs = pd.read_csv(f'{base_dir}/{folder}/reqs.csv')
      offices = pd.read_csv(f'{base_dir}/{folder}/offices.csv')
      dist = pd.read_csv(f'{base_dir}/{folder}/distance_matrix.csv')

      start_time = time.time()
      solution = solve_transportation_problem(reqs, offices, dist)
      end_time = time.time()

      if solution:
          save_solution(solution, f'solution_{folder}.csv')
          print(f"Стоимость для {folder}: {solution['objective']:.2f}, время: {end_time-start_time:.1f}с")

if __name__ == "__main__":
    run_for_all_folders()

Стоимость для 10_nodes: 83784113565.50, время: 0.1с
Стоимость для 20_nodes: 1653797853888.06, время: 12.9с
Стоимость для 50_nodes: 29136162009441.45, время: 973.1с


Решаем задачу по частям. Начинаем с небольшого набора путей, находим оптимальное их использование, затем ищем новые выгодные пути и повторяем.

Пошагово:
Создаем кэши стоимостей и пропускных способностей
Генерируем начальные пути: только прямые и через 1 промежуточный узел

Основной цикл:

 * Решаем ограниченную задачу - находим оптимальное распределение груза по текущим путям

 * Из решения получаем "ценности" ресурсов (двойственные переменные)

 * Ищем новые пути, которые дешевле с учетом этих "ценностей"

 * Если нашли выгодные пути - добавляем их и повторяем цикл

Путь выгоден, если его полная стоимость МИНУС "ценность" использованных ресурсов < 0
Ресурсы, за которые "платим":Пропускная способность ТС на ребрах,Пропускная способность узлов, Выполнение заявок